# Peleando con los datos

El objetivo de este modulo es aprender a cargar datos, limpiarlos, y manipularlos. Para esto, usaremos dos archivos ("stations.csv" y "trips.csv") que corresponden a datos de una empresa de arriendo de bicicletas (como BikeSantiago). Los datos tiene las estaciones y los viajes registrados entre el 2011 y el 2012 (+ de 550.000 viajes!)

Para esto, usaremos el paquete `Dataframes`

In [ ]:
using DataFrames
using CSV

Primero cargamos los datos (Ojo, los datos vienen con un registro "NA" para aquellos que no son validos):

In [ ]:
dataStations = CSV.read("stations.csv",missingstring="NA")

In [ ]:
#dataTrips = CSV.read("trips.csv",missingstring="NA",rows_for_type_detect=100000,dateformat="yyyy-mm-dd HH:MM:SS-ss")
dataTrips = CSV.read("trips_new.csv",rows_for_type_detect=100000)

Con esto, podemos pedirle que nos describa los datos, utilizando el comando `describe`:

In [ ]:
describe(dataStations)

In [ ]:
describe(dataTrips, stats=:all)

Podemos ver el numero de registros del DataFrame

In [ ]:
nrow(dataTrips)

O pedir una columna en específica

In [ ]:
dataTrips[:zip_code]

También podemos pedir un subconjunto de elementos, como si fuera una matriz. Por ejemplo, las 10 primeras filas.

In [ ]:
dataTrips[1:10,:]

O las filas que cumplen con una condición. Por ejemplo, los viajes de la bicicleta B00468.

In [ ]:
dataTrips[dataTrips[:bike_nr] .== "B00468", :]

Ahora, transformemos las duraciones (que están en segundos) en minutos.

In [ ]:
dataTrips[:duration] = map(x -> x/60, dataTrips[:duration])

In [ ]:
describe(dataTrips, stats=:all)

In [ ]:
nrow(dataTrips)

Fijemonos que tenemos varias columnas con datos faltantes.  Algunos no son graves (*zip_code*, *birth_date*) pero otros pueden ser errores (*start_station* y *end_station*).  Podemos eliminar las final con datos faltantes `missing` usnado las manipulaciones anteriores.

Por ejemplo, podemos ver si un dato es `missing` o no, con el comando `ismissing()`.  Esto hay que aplicarlo sobre ***CADA*** registro de la columna (`broadcast`), y con eso podemos filtrar los datos que queremos.

In [ ]:
ismissing.(dataTrips[:start_station])

In [ ]:
finalData = dataTrips[!ismissing.(dataTrips[:start_station]) & !ismissing.(dataTrips[:end_station]),:]


In [ ]:
nrow(finalData)

Grafiquemos las duraciones. Para eso, usaremos el paquete `Gadfly`, que estudiaremos con mas detalle en el siguiente módulo.

In [ ]:
using Gadfly

In [ ]:
plot(finalData, x=:duration, Geom.histogram(bincount=20))

¿Que pasó?  Hay outliers en los datos... Limitemonos entonces a aquellos datos con duraciones menores a 1 dia.

In [ ]:
plot(finalData[finalData[:duration].< 24*60,:],x=:duration, Geom.histogram(bincount=100))

Mas razonable. Actualicemos los datos solo a aquellos viajes que duran menos de 24 horas.

In [ ]:
finalData = finalData[finalData[:duration] .< 24*60,:]

In [ ]:
nrow(finalData)

Usemos las fechas. Estas pertenecen al tipo `DateTime`, por lo que podemos hacerles preguntas a ese registro, como el día de la semana de cada fecha, o el mes, etc.   <br>

Partamos analizando las fechas en que se realizan los viajes, y los dias de semana que corresponden.

In [ ]:
date = map(df -> Dates.Date(df),finalData[:start_date])

In [ ]:
plot(finalData, x=:start_station, Geom.histogram)

In [ ]:
plot(x=map(x -> Dates.dayname(x),finalData[:start_date]), Geom.histogram)

# Analizando los datos. 
Ok, tenemos los datos ya limpios. Esto es generlamente muy tedioso, pero vimos que tenemos herramientas para poder hacerlo de forma un poco mas sencilla. <br>
Ahora queremos empezar a hacer algunos análisis. Para esto, utilizaremos algunas funciones propias de los `Dataframes`:

## by
`by` nos permite analizar un dataframe, bajo el paradigma Split-Apply-Combine.  Este comando usa los parametros:
```
by(DataFrame,Columna,funcion)
```
con lo cual, separa (*split*) los datos del `Dataframe` para los distintos valores de `columna`, y a cada grupo de ellos les aplica (*apply*) la función `function`, combinando (*combine*) el resultado en una tabla. 

`by` es el equivalente de utilizar la secuencia de comandos `combine(map(f, groupby(d, cols)))`

In [ ]:
groupby(finalData,:start_station)

In [ ]:
map(df -> nrow(df),groupby(finalData,:start_station))

In [ ]:
combine(map(df -> nrow(df),groupby(finalData,:start_station)))

In [ ]:
by(finalData,:start_station,nrow)

Por ejemplo, podemos querer contar cuantos suscriptores de cada tipo hay. PAra esto, agrupamos los datos por el tipo de suscripción, y cada grupo de ellos los contamos. Es decir:

In [ ]:
by(finalData,:subscription_type,nrow)

También podemos aplicar un `by` a un conjiunto de columnas

In [ ]:
by(finalData,[:start_station,:end_station],nrow)

**Pregunta**: ¿Cuál es la duración promedio de los viajes para cada tipo de suscriptor?

**PREGUNTA 1**: ¿Cual es la duración promedio por género?

**PREGUNTA 2**: ¿Cual es la suma de las duraciones por género?

**PREGUNTA 3**: ¿Cuál es la duración promedio para cada día de la semana? (usemos la fecha de inicio del viaje)

**PREGUNTA 4**: ¿Cuál es la duración promedio para cada mes del año? (usemos la fecha de término del viaje)

**Pregunta BONUS**: ¿Cuál es la proporción de usuarios `Casual` para cada estación de inicio? Cuales son las estaciones con mayor y menor proporción de usuarios casuales?

Supongamos ahora que queremos rescatar los dos días mas utilizados, por cada tipo de usuario, o por estación. Esto requeriría crea una propia función, que nos entregue los dos dias. Pero esto sabemos hacerlo!


In [ ]:
finalData[:startDay]=map(x -> Dates.dayname(x),finalData[:start_date])

In [ ]:
sort(by(finalData,:startDay,nrow),:x1,rev=true)[1:2,:startDay]

In [ ]:
function best2days(df)
    tmp = sort(by(df,:startDay,nrow),:x1,rev=true)
    return tmp[1,:startDay] * " " * tmp[2,:startDay]
end

In [ ]:
by(finalData,:subscription_type,df -> best2days(df))

**Ejercicio**: La empresa tiene una tarifa distinta para aquellos viajes que duran mas de media hora.   Por eso, Calcule la proporción de viajes desde cada estación de inicio (*start_station*) que duran mas de 30 minutos. 

**Ejercicio (BONUS):** encuentre el tipo de suscripción más común para cada estación de partida.   

**Ejercicio**: Cree un nuevo `DataFrame` de bicicletas, donde cada fila tenga:
- El id de cada bicicleta `bike_nr`
- la duración promedio de cada bicicleta
- la desviación estándar de las duraciones de cada bicicleta
- el número de viajes realizados en cada bicicleta

Para esto, podemos hacer que el resultado de la función a aplicar sobre cada grupo sea un `Dataframe`, por lo que al combinarlo tendremos el dataframe pedido.   Para definir un DataFrame usamos el comando `DataFrame`:
```
DataFrame(numeroFilas = nrow(finalData), duracion_promedio = mean(finalData[:duration]))
```

**Ejercicio (BONUS)**: Incluya las siguientes variables para cada fila:
- número de viajes que parten y terminan en un día diferente
- estación mas común de inicio de un viaje 
- estación mas común de término de un viaje

# Agrupando datos de diferentes tablas

Podemos unir datos de distintas tablas, que nos permitan usar información de ellas. Por ejemplo, si quisieramos aproximar la distancia de cada viaje, podríamos usar la latitud y longitud de las estaciones de inicio.  Esto requeriría "unir" (`join`) ambas tablas, en una sola, agregando a la tabla de viajes los datos de la estación de inicio.  Para esto, usamos el comando `join`.

In [ ]:
merged = join(finalData,dataStations, on = (:start_station,:id))

In [ ]:
merged = join(merged,dataStations, on = (:end_station,:id))

Para calcular la distancia entre dos puntos geo-referenciados, usaremos el paquete `Geodesy` que permite hacer esto:

In [ ]:
using Geodesy

In [ ]:
merged[:geo_dist] = map((lat1,lon1,lat2,lon2) -> distance(LLA(lat1,lon1,0.0),LLA(lat2,lon2,0))/1000.0, merged[:lat],merged[:lng],merged[:lat_1],merged[:lng_1])

In [ ]:
merged

In [ ]:
plot(merged, x=:geo_dist, Geom.histogram)

# Ejercicio Final:
Hubway cobra basado en la duración del viaje. Usuarios *Casual* pagan `$0` si el viaje dura menos de 30 minutos, `$2` si dura entre 30 y 60 minutos, y `$6` para viajes entre 60 a 90 minutos Por cada 30 minutos adicioanles, se cobran `$8` adicionales, con un máximo de `$100`.   Un usuario *Registered* para el 75\% de la tarifa solamente (con un máximo de `$75`, por lo tanto). 

**Ejercicio**: Agregue una columna fee que indique cuanto se cobró en cada viaje.